In [1]:
from binance.client import Client
from binance.enums import *
from keras.engine import sequential
from keras.layers.recurrent import RNN
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, LSTM, Dropout, Bidirectional, GRU, ConvLSTM2D, Activation, CuDNNGRU, CuDNNLSTM, Conv2D, Conv1D, Input, MaxPooling1D, \
                         GlobalMaxPooling1D, Embedding, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, \
                                       EarlyStopping, LearningRateScheduler, \
                                       ReduceLROnPlateau, CSVLogger
from tensorflow.keras.callbacks.experimental import BackupAndRestore
from tensorflow.keras.metrics import Accuracy, BinaryAccuracy, \
                                     CategoricalAccuracy, \
                                     SparseCategoricalAccuracy, \
                                     TopKCategoricalAccuracy, \
                                     SparseTopKCategoricalAccuracy

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import math
import keras
import sys 
import pandas as pd
import numpy as np
import json
import os

2021-11-09 00:51:27.209940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-09 00:51:27.210009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Time Distributed Layer
This wrapper allows to apply a layer to every temporal slice of an input.<br>

Every input should be at least 3D, and the dimension of index one of the first input will be considered to be the temporal dimension.<br>

## Example
Consider a batch of 32 video samples, where each sample is a 128x128 RGB image with channels_last data format, across 10 timesteps. The batch input shape is (32, 10, 128, 128, 3).<br>

You can then use TimeDistributed to apply the same Conv2D layer to each of the 10 timesteps, independently.<br>

Because TimeDistributed applies the same instance of Conv2D to each of the timestamps, the same set of weights are used at each timestamp.<br>

In [2]:
inputs = tf.keras.Input(shape=(10, 128, 128, 3))
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
outputs = tf.keras.layers.TimeDistributed(conv_2d_layer)(inputs)
outputs.shape

2021-11-09 01:03:29.216208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-09 01:03:29.216237: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-09 01:03:29.216257: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aspire): /proc/driver/nvidia/version does not exist
2021-11-09 01:03:29.216559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([None, 10, 126, 126, 64])

## Example 2
### Without Time Distributed
In this section, we develop an LSTM to output the sequence all at once, although without the TimeDistributed wrapper layer.<br>

The input for LSTMs must be three dimensional. We can reshape the 2D sequence into a 3D sequence with 1 sample, 5 time steps, and 1 feature. We will define the output as 1 sample with 5 features.<br>

Immediately, you can see that the problem definition must be slightly adjusted to support a network for sequence prediction without a TimeDistributed wrapper. Specifically, output one vector rather build out an output sequence one step at a time. The difference may sound subtle, but it is important to understanding the role of the TimeDistributed wrapper.<br>

In [3]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length)])
X = seq.reshape(1, length, 1)
y = seq.reshape(1, length)
# define LSTM configuration
n_neurons = length
n_batch = 1
n_epoch = 500
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(length, 1)))
model.add(Dense(length))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train LSTM
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=2)
# evaluate
result = model.predict(X, batch_size=n_batch, verbose=0)
for value in result[0,:]:
	print('%.1f' % value)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5)                 140       
_________________________________________________________________
dense (Dense)                (None, 5)                 30        
Total params: 170
Trainable params: 170
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500


2021-11-09 01:08:57.290179: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 - 2s - loss: 0.2314
Epoch 2/500
1/1 - 0s - loss: 0.2286
Epoch 3/500
1/1 - 0s - loss: 0.2257
Epoch 4/500
1/1 - 0s - loss: 0.2229
Epoch 5/500
1/1 - 0s - loss: 0.2201
Epoch 6/500
1/1 - 0s - loss: 0.2173
Epoch 7/500
1/1 - 0s - loss: 0.2146
Epoch 8/500
1/1 - 0s - loss: 0.2118
Epoch 9/500
1/1 - 0s - loss: 0.2090
Epoch 10/500
1/1 - 0s - loss: 0.2062
Epoch 11/500
1/1 - 0s - loss: 0.2035
Epoch 12/500
1/1 - 0s - loss: 0.2007
Epoch 13/500
1/1 - 0s - loss: 0.1980
Epoch 14/500
1/1 - 0s - loss: 0.1953
Epoch 15/500
1/1 - 0s - loss: 0.1926
Epoch 16/500
1/1 - 0s - loss: 0.1899
Epoch 17/500
1/1 - 0s - loss: 0.1872
Epoch 18/500
1/1 - 0s - loss: 0.1845
Epoch 19/500
1/1 - 0s - loss: 0.1818
Epoch 20/500
1/1 - 0s - loss: 0.1792
Epoch 21/500
1/1 - 0s - loss: 0.1765
Epoch 22/500
1/1 - 0s - loss: 0.1739
Epoch 23/500
1/1 - 0s - loss: 0.1713
Epoch 24/500
1/1 - 0s - loss: 0.1686
Epoch 25/500
1/1 - 0s - loss: 0.1660
Epoch 26/500
1/1 - 0s - loss: 0.1635
Epoch 27/500
1/1 - 0s - loss: 0.1609
Epoch 28/500
1/1 - 0s 

The LSTM units have been crippled and will each output a single value, providing a vector of 5 values as inputs to the fully connected layer. The time dimension or sequence information has been thrown away and collapsed into a vector of 5 values.<br>

The sequence is reproduced correctly, but as a single piece rather than stepwise through the input data. We may have used a Dense layer as the first hidden layer instead of LSTMs as this usage of LSTMs does not take much advantage of their full capability for sequence learning and processing.

### With Timedistributed
- **The input must be (at least) 3D.** This often means that you will need to configure your last LSTM layer prior to your TimeDistributed wrapped Dense layer to return sequences (e.g. set the “return_sequences” argument to “True”).
- **The output will be 3D.** This means that if your TimeDistributed wrapped Dense layer is your output layer and you are predicting a sequence, you will need to resize your y array into a 3D vector.

The single output value in the output layer is key. It highlights that we intend to output one time step from the sequence for each time step in the input. It just so happens that we will process 5 time steps of the input sequence at a time.<br>

The TimeDistributed achieves this trick by applying the same Dense layer (same weights) to the LSTMs outputs for one time step at a time. In this way, the output layer only needs one connection to each LSTM unit (plus one bias).<br>

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length)])
X = seq.reshape(1, length, 1)
y = seq.reshape(1, length, 1)
# define LSTM configuration
n_neurons = length
n_batch = 1
n_epoch = 1000
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(length, 1), return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train LSTM
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=2)
# evaluate
result = model.predict(X, batch_size=n_batch, verbose=0)
for value in result[0,:,0]:
	print('%.1f' % value)

The fully connected output layer is a very different story. In fact, it matches the one-to-one example exactly. One neuron that has one weight for each LSTM unit in the previous layer, plus one for the bias input.<br>

This does two important things:
- Allows the problem to be framed and learned as it was defined, that is one input to one output, keeping the internal process for each time step separate.
- Simplifies the network by requiring far fewer weights such that only one time step is processed at a time.

The one simpler fully connected layer is applied to each time step in the sequence provided from the previous layer to build up the output sequence.<br>